In [32]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass

!pip install tensorflow-datasets
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Sequential
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow.keras.preprocessing import text

In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-04-25 17:40:00--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.05s   

2025-04-25 17:40:00 (6.41 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2025-04-25 17:40:00--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [35]:
column_names = ["type", "message"]

train_file = pd.read_csv(train_file_path, sep="\t", names=column_names)
train_file["type"] = train_file["type"].astype("category").cat.codes

test_file = pd.read_csv(test_file_path, sep="\t", names=column_names)
test_file["type"] = test_file["type"].astype("category").cat.codes

train_file.head(), test_file.head()

(   type                                            message
 0     0  ahhhh...just woken up!had a bad dream about u ...
 1     0                           you can never do nothing
 2     0  now u sound like manky scouse boy steve,like! ...
 3     0  mum say we wan to go then go... then she can s...
 4     0  never y lei... i v lazy... got wat? dat day ü ...,
    type                                            message
 0     0  i am in hospital da. . i will return home in e...
 1     0         not much, just some textin'. how bout you?
 2     0  i probably won't eat at all today. i think i'm...
 3     0  don‘t give a flying monkeys wot they think and...
 4     0                                who are you seeing?)

In [37]:
train_file = Dataset.from_tensor_slices((train_file["message"].values, train_file["type"].values))
train_file = train_file.shuffle(100).batch(32).prefetch(AUTOTUNE)

test_file = Dataset.from_tensor_slices((test_file["message"].values, test_file["type"].values))
test_file = test_file.batch(32).prefetch(AUTOTUNE)

In [38]:
vectorizer = layers.TextVectorization(max_tokens=1000, output_mode="int", output_sequence_length=1000)
vectorizer.adapt(train_file.map(lambda text, label: text))

2025-04-25 17:57:56.561167: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [40]:
model = Sequential([
    vectorizer,
    layers.Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=64, mask_zero=True),
    layers.GlobalAveragePooling1D(),
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(train_file, test_file, epochs=5, batch_size=32)

ValueError: When providing `x` as a tf.data.Dataset, `y` should not be passed. Instead, the targets should be included as part of the tf.data.Dataset.

In [62]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_prob = model.predict([pred_text])[0][0]
    
    return [pred_prob, "spam" if pred_prob >= 0.5 else "ham"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

ValueError: Unrecognized data type: x=['how are you doing today?'] (of type <class 'list'>)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
